In [3]:
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [4]:
def getStateAtMoment(base,distance,sinal,inst):
    
    #    'Instante' de saída
    #            |
    #            v
    new_inst = inst+(distance/sinal)
    #           |______________|
    #                  |
    #                  |
    #         Quantidade de 'instantes'
    #         que demora para cruzar o
    #              arco atual

    #Aqui verificamos se pelo menos
    #um 'instante' adicional foi 
    #preciso para percorrer o arco.
    if(new_inst-inst>=1):
         #Exemplo:
        #     Instante que entrou no arco = 7
        #     
        #     Instante que saiu do arco = 9.3451
        #
        #     N° de instantes para cruzar o arco = 2.3451
        #
        #     0.3451 é maior que 1, por isso arredondamos
        #     9.3451 para 9 que é o instante em que ele está
        #     cruzando o arco e guardamos a diferença que
        #     representa o número de instantes que demora para
        #     cruzar o arco.
        diff=new_inst-inst
        
    else:
        #Exemplo:
        #     Instante que entrou no arco = 7
        #     Instante que saiu do arco = 7.3451
        #     N° de instantes para cruzar o arco = 0.3451
        #     0.3451 é menor que 1, portanto o veículo 
        #     cruzou o arco antes do sinal mudar de estado,
        #     então retornarmos o estado inicial do sinal
        #     e o instante de saída do arco
        return base, new_inst
    
    #Se a diferença for par, nesse caso
    #o estado do sinal que ele encontrará
    #quando estiver no final do arco será
    #igual ao estado do sinal quando ele
    #entrou, por exemplo:
    #0 -> estado do sinal do final do arco quando entrou
    #
    #  i+1   i+3
    #   v     v
    #0->1->0->1->0...
    #^     ^     ^
    #i    i+2   i+4
    if round(diff)%2==0:
        return base,new_inst
    else: 
        #Se for impar, nós
        #alternamos o estado
        #de entrada
        base = not base
        return base,new_inst

In [5]:
def DecomposeRoute(Solution,veiculos,vitimas):
    #Rota percorrida por cada veículo
    routes = {x:{} for x in veiculos.index}
    
    #Separa as rotas de cada veículo
    for i in Solution:
            #
            #
            #"Arc_i_j_k"
            #     |   
            #     v                      0   1   2
            aux = i.split("_")[1:] #<- ["i","j","k"]
            #        ^
            #        |
            #["Arc","i","j","k"]
            #
            #         veiculo=k    saida=i      chegada=j
            #             |            |            |
            #             v            v            v
            routes[int(aux[2])][int(aux[0])]=int(aux[1])
            # ^
            # |_ Rotas agrupadas por veículo
    
    #Rotas ordenadas de acordo com
    #a sequência dos arcos
    ordered_routes = {x:[] for x in veiculos.index} 
    
    for k in veiculos.index:
        
        #Primeiramente, verificamos se o veículo 'k'
        #percorreu, ou não, pelo menos um arco
        if len(routes[k]) > 0:
            
            #Se ele percorreu pelo menos um arco:
            #Começamos verificando qual o ponto de saida
            #do veículo 'k' e atribuímos esse nodo para
            #o ponto de partida do nosso loop
            
            inicial = veiculos.loc[k]["Inicial"]
            #  ^
            #  |
            #Saída de 'k'
            #
            #Chegada de 'k'
            #saindo do ponto
            #inicial
            #  |
            #  v
            actual_node = routes[k].pop(inicial)
            
            #Adiciona o nodo inicial ao começo
            #da lista de arcos
            ordered_routes[k].append(inicial)
            
            if len(routes[k]) == 0:
                #Se a rota só tiver um arco
                #adicionamos o nodo seguinte
                #e então finalizamos essa rota
                ordered_routes[k].append(actual_node)
            else:
                #Se a rota tiver mais arcos,
                #prosseguimos adicionando os
                #nodos correspondentes na lista
                while actual_node not in vitimas["Ponto"].values:
                    #                     Nodo de chegada do
                    #                       arco anterior
                    #                             |
                    #                             v
                    ordered_routes[k].append(actual_node)
                    
                    #Nodo de chegada        Torna-se o nodo
                    #do arco que começa     de saída para o
                    #em actual_node            arco atual
                    #   |                          |
                    #   v                          v
                    next_node = routes[k].pop(actual_node)
                    #   ^
                    #   |
                    #   \           Transformamos actual_node
                    #   ----------  no nodo de chegada atual
                    #             \ e então repetimos o processo
                    #             |
                    #             v
                    actual_node=next_node
                
                #Adiciona o último nodo da rota, pois
                #o while termina antes de adicioná-lo
                ordered_routes[k].append(actual_node)
    return ordered_routes

In [6]:
def makeArcCount(ordered_routes,_DistanceMatrix):
    arc_count = {}

    for k in ordered_routes.keys():
        
        for i in range(len(ordered_routes[k])-1):
            saida = ordered_routes[k][i]
            chegada = ordered_routes[k][i+1]

            if (saida,chegada) not in list(arc_count.keys()):
                arc_count[(saida,chegada)]= [k]
            else:
                arc_count[(saida,chegada)].append(k)
        
    return arc_count

In [2]:
def buildSets(_DistanceMatrix,rota_k,sinal,prioridade,pontos,estado,arcos_fixos,nodo=0,instante=0,maior_instante=0,sinais_alterados=[],matriz_sinais=[]):
    if nodo == (len(rota_k)-1):
        #Configuração atual de sinais
        config_atual={}
        
        #Soma dos pesos dessa configuração
        peso_rota=0
        
        for i in sinais_alterados:
            arco = i["arco"]         #<-(saida,chegada)
            alterado = i["alterado"] #<-(0,1)
            instante = i["instante"] #<-(t)
            peso = i["peso"]         #<-Wij

            
            #Adiciona (saida,chegada):{0/1,t} na matriz de sinais
            #                    0=Não alterado
            #                    1=Alterado
            #                    t=Instante
            config_atual[arco]=(alterado,instante)
            
            #Adiciona o peso do arco atual ao peso total da rota
            peso_rota+=peso
        
        #Multiplica o peso da rota 
        #pelo instante de chegada na vítima
        peso_rota*=prioridade
            
        #Adiciona a configuração atual de sinais e o peso correspondente
        #à matriz de configurações para a rota atual
        matriz_sinais.append({"sinais":config_atual,"peso":peso_rota})
        
        #Retorna a matriz de sinais, agora
        #com a configuração atual de sinais
        return matriz_sinais,maior_instante
    
    else:
        #Nodo de saída do arco
        saida = rota_k[nodo]
        #Nodo de chegada do arco
        chegada = rota_k[nodo+1]
        
        #Custo básico 
        custo = _DistanceMatrix.loc[saida][chegada]
        
        #Esse trecho pega o índice do nodo de chegada
        #para verificar o estado do arco (saida,chegada)
        index = pontos.loc[saida][2:][pontos.loc[saida][2:] == chegada].index[0]
        base = estado.loc[saida][index]
        
        #Se o estado do sinal for 
        #alterado, pagamos uma 
        #penalidade pelo
        #número de sinais
        #que precisam ser
        #alterados que não
        #estão na rota
        #     |
        #     v
        alfa=(2*sinal)/(prioridade) 
        
        estado1,instante1 = getStateAtMoment(base,custo,sinal,instante)
        #                     {estado1=0, se ligado no instante   }
        #             estado1={estado1=1, se desligado no instante}
        #               |
        #               v                       ------------------
        peso1 = custo+estado1*sinal# <- sinal = {tempo pra o sinal}
        #                                       {mudar de estado  }
        #                                       -------------------
        #
        # Se estado1=0 -> estado1*sinal=sinal, então pagamos o custo de esperar no sinal
        # Se estado1=1 -> estado1*sinal=0, o sinal está verde e não precisamos esperar
        
        estado2,instante2 = getStateAtMoment(base,custo+sinal,sinal,instante)
        #                                ----------------------
        #                                {Negação do estado2  }
        #                    (1-estado2)={Se estado2=0, vira 1}
        #                         |      {Se estado2=1, vira 0}
        #                         |       ---------------------
        #                         v                        ------------------
        peso2 = custo+alfa+(1-estado2)*sinal # <- sinal = {tempo pra o sinal}
        #               ^                                 {mudar de estado  }
        #               |    -----------------            ------------------
        #             alfa = {Penalidade por }            
        #                    {alterar o sinal}
        #                    ----------------
        # Se (1-estado2)=1 -> (1-estado2)*sinal=0, então pagamos o custo de esperar no sinal
        # Se (1-estado2)=0 -> (1-estado2)*sinal=sinal, o sinal está verde e não precisamos esperar
        
        if len(sinais_alterados) == 0:
            sinais1=[{"arco":(saida,chegada),"alterado":0,"peso":peso1,"instante":instante1}]
            sinais2=[{"arco":(saida,chegada),"alterado":1,"peso":peso2,"instante":instante2}]
        else:

            sinais1 = sinais_alterados.copy()
            sinais2 = sinais_alterados.copy()
            
            sinais1.append({"arco":(saida,chegada),"alterado":0,"peso":peso1,"instante":instante1})
            sinais2.append({"arco":(saida,chegada),"alterado":1,"peso":peso2,"instante":instante2})
            
        #Esse trecho vai garantir que
        #a gente vai receber no final
        #o maior instante, necessário
        #para criarmos a matriz de de-
        #cisão do mesmo tamanho.
        if instante1 > maior_instante:
            maior_instante = instante1
            
        elif instante2 > maior_instante:
            maior_instante=instante2
        
        
        #A intenção desse programa é popular
        #primeiro por profundidade, os ramos
        #à esquerda são ramos onde a alteração
        #do sinal é 0, e à direita 1
        if (saida,chegada not in arcos_fixos):
            #Cairá nessa alternativa quando o arco não
            #tiver sido fixado anteriormente numa 
            #solução do modelo de Sinais
            matriz_sinais, maior_instante = buildSets(_DistanceMatrix,rota_k,sinal,prioridade,pontos,estado,{},nodo+1,instante1,maior_instante,sinais1,matriz_sinais)
            # Comportamento esperado:
            #                       
            #               o
            #              / \
            #             o aberto
            #            / \
            # Entrando<-x  aberto
            #          /
            #        ...
            # matriz_sinais->matriz_sinais + configuração 1

            matriz_sinais, maior_instante = buildSets(_DistanceMatrix,rota_k,sinal,prioridade,pontos,estado,{},nodo+1,instante2,maior_instante,sinais2,matriz_sinais)
            # Comportamento esperado:
            #            o
            #           / \
            #          o
            #         / \
            # Saiu <-v  x->Entrando
            #       /
            #     ...
            # matriz_sinais->matriz_sinais + configuração 1 + configuração 2
        elif arcos_fixos[(saida,chegada)]["alterado"] == 0 and arcos_fixos[(saida,chegada)]["instante"]==instante1:
            #Cairá nessa alternativa quando o
            #arco tiver sido usado, mas não
            #alterado, em uma das soluções
            #do modelo de Sinais
            matriz_sinais, maior_instante = buildSets(_DistanceMatrix,rota_k,sinal,prioridade,pontos,estado,{},nodo+1,instante1,maior_instante,sinais1,matriz_sinais)
            # Comportamento esperado:
            #                       
            #               o
            #              / \
            #             o aberto
            #            / \
            # Entrando<-x  aberto
            #          /
            #        ...
            # matriz_sinais->matriz_sinais + configuração 1
        elif arcos_fixos[(saida,chegada)]["alterado"] == 1 and arcos_fixos[(saida,chegada)]["instante"]==instante2:
            #Cairá nessa alternativa quando o
            #arco tiver sido usado e tiver
            #sido alterado, em uma das 
            #soluções do modelo de Sinais
            matriz_sinais, maior_instante = buildSets(_DistanceMatrix,rota_k,sinal,prioridade,pontos,estado,{},nodo+1,instante2,maior_instante,sinais2,matriz_sinais)
            # Comportamento esperado:
            #            o
            #           / \
            #          o
            #         / \
            # Saiu <-v  x->Entrando
            #       /
            #     ...
            # matriz_sinais->matriz_sinais + configuração 1 + configuração 2
        else:
            #Cairá nessa alternativa quando o
            #arco tiver sido usado, sendo alterado
            #ou não, em uma das soluções do modelo
            #de sinais, porém em um instante diferente
            #da alteração naquele respectivo modelo
            matriz_sinais, maior_instante = buildSets(_DistanceMatrix,rota_k,sinal,prioridade,pontos,estado,{},nodo+1,instante1,maior_instante,sinais1,matriz_sinais)
            # Comportamento esperado:
            #                       
            #               o
            #              / \
            #             o aberto
            #            / \
            # Entrando<-x  aberto
            #          /
            #        ...
            # matriz_sinais->matriz_sinais + configuração 1

            matriz_sinais, maior_instante = buildSets(_DistanceMatrix,rota_k,sinal,prioridade,pontos,estado,{},nodo+1,instante2,maior_instante,sinais2,matriz_sinais)
            # Comportamento esperado:
            #            o
            #           / \
            #          o
            #         / \
            # Saiu <-v  x->Entrando
            #       /
            #     ...
            # matriz_sinais->matriz_sinais + configuração 1 + configuração 2
        return matriz_sinais,maior_instante